In [1]:
from samples.frontend import CacheConfig, in_out_nodes_generation, inst_block_gen, extract_mem_blocks
from samples.fixpoint import Fixpoint, FixpointType, fixpoint_nodes_generation
from pprint import pprint

In [2]:
"""
    从CFG到cache analysis的接口 需要输入五个信息
    有关CFG的信息为:
        1. 所有 basic block 的标识 相当于其identity 必须是唯一的 这里直接用我们之前使用的 basic block的首指令地址, 相当于blockHead
        2. 入口 basic block , 也就是整个CFG的头, 这里可以理解为main函数的开始 相当于blockHead[1]
        3. 边信息 这里是有向边 从左指向右 ---> 构成一个元组 有几条边就有几个元组 所有元组构成一个总的边列表
    有关cache配置的信息：
        1. 只要知道地址中的偏移位和set_index位以及cache是几路相联即可 or 输入直接得到的信息也可以 比如cache总大小, cache line 的大小, 以及几路相联 也能根据关系计算出来 cache line size ---> offset_len;
    有关访存的信息：
        1. 需要输入每个 basic block 所访存的地址信息, 比如指令的则是 开始 - 结尾, dict类型, basic block identity ---> key ; tuple(start,end) ---> value
"""

# Idents of all nodes.
all_nodes = ['400604', '400630', '40066c', '400708', '400724', '400730', '40073c', '400740', '40074c', '400754', '400758', '400770', '40079c', '4007c0', '4007d4', '4007fc', '40080c', '400848', '400864', '400870', '400874']
entrynode = '400604'
# All directed edges. The CFG is determined by nodes and edges.
edges = [('400604', '400740'), ('400604', '400630'), ('400630', '400730'), ('40066c', '400724'), ('40066c', '400708'), ('400708', '400874'), ('400724', '400730'), ('400730', '40066c'), ('400730', '40073c'), ('40073c', '400870'), ('400740', '400754'), ('400740', '40074c'), ('40074c', '400758'), ('400754', '400758'), ('400758', '400864'), ('400770', '4007c0'), ('400770', '40079c'), ('40079c', '400848'), ('4007c0', '4007fc'), ('4007d4', '4007fc'), ('4007fc', '4007d4'), ('4007fc', '40080c'), ('40080c', '400848'), ('400848', '400864'), ('400864', '400770'), ('400864', '400870'), ('400870', '400874')]
# For instruction cache analysis, the instruction range of each basic block.
address_range = {'400604': (4195844, 4195884), '400630': (4195888, 4195944), '40066c': (4195948, 4196100), '400708': (4196104, 4196128), '400724': (4196132, 4196140), '400730': (4196144, 4196152), '40073c': (4196156, 4196156), '400740': (4196160, 4196168), '40074c': (4196172, 4196176), '400754': (4196180, 4196180), '400758': (4196184, 4196204), '400770': (4196208, 4196248), '40079c': (4196252, 4196284), '4007c0': (4196288, 4196304), '4007d4': (4196308, 4196344), '4007fc': (4196348, 4196360), '40080c': (4196364, 4196420), '400848': (4196424, 4196448), '400864': (4196452, 4196460), '400870': (4196464, 4196464), '400874': (4196468, 4196472)}

# Cache config.
cache_conf = CacheConfig(offset_len=6, set_index_len=8, assoc=4)

In [3]:
""" 获取每个 basic block 的 in_nodes 和 out_nodes (如果没有入 没有出 则标记位set() ) """

in_nodes, out_nodes = in_out_nodes_generation(all_nodes, edges)

pprint(in_nodes)
pprint(out_nodes)

{'400604': set(),
 '400630': {'400604'},
 '40066c': {'400730'},
 '400708': {'40066c'},
 '400724': {'40066c'},
 '400730': {'400630', '400724'},
 '40073c': {'400730'},
 '400740': {'400604'},
 '40074c': {'400740'},
 '400754': {'400740'},
 '400758': {'40074c', '400754'},
 '400770': {'400864'},
 '40079c': {'400770'},
 '4007c0': {'400770'},
 '4007d4': {'4007fc'},
 '4007fc': {'4007d4', '4007c0'},
 '40080c': {'4007fc'},
 '400848': {'40080c', '40079c'},
 '400864': {'400758', '400848'},
 '400870': {'400864', '40073c'},
 '400874': {'400708', '400870'}}
{'400604': {'400630', '400740'},
 '400630': {'400730'},
 '40066c': {'400708', '400724'},
 '400708': {'400874'},
 '400724': {'400730'},
 '400730': {'40073c', '40066c'},
 '40073c': {'400870'},
 '400740': {'40074c', '400754'},
 '40074c': {'400758'},
 '400754': {'400758'},
 '400758': {'400864'},
 '400770': {'4007c0', '40079c'},
 '40079c': {'400848'},
 '4007c0': {'4007fc'},
 '4007d4': {'4007fc'},
 '4007fc': {'4007d4', '40080c'},
 '40080c': {'400848'},
 

In [4]:
""" basic block 到 memory block 的映射; 每个 basic block 访问了 那些 memory block; 且 memory block 由 tag + set_index 唯一确定 """

mem_block_dict = {ident: inst_block_gen(*address_range[ident], cache_config=cache_conf) for ident in all_nodes}
pprint(mem_block_dict)

{'400604': [<MemBlock tag:0x100 set_index:24>],
 '400630': [<MemBlock tag:0x100 set_index:24>,
            <MemBlock tag:0x100 set_index:25>],
 '40066c': [<MemBlock tag:0x100 set_index:25>,
            <MemBlock tag:0x100 set_index:26>,
            <MemBlock tag:0x100 set_index:27>,
            <MemBlock tag:0x100 set_index:28>],
 '400708': [<MemBlock tag:0x100 set_index:28>],
 '400724': [<MemBlock tag:0x100 set_index:28>],
 '400730': [<MemBlock tag:0x100 set_index:28>],
 '40073c': [<MemBlock tag:0x100 set_index:28>],
 '400740': [<MemBlock tag:0x100 set_index:29>],
 '40074c': [<MemBlock tag:0x100 set_index:29>],
 '400754': [<MemBlock tag:0x100 set_index:29>],
 '400758': [<MemBlock tag:0x100 set_index:29>],
 '400770': [<MemBlock tag:0x100 set_index:29>,
            <MemBlock tag:0x100 set_index:30>],
 '40079c': [<MemBlock tag:0x100 set_index:30>],
 '4007c0': [<MemBlock tag:0x100 set_index:31>],
 '4007d4': [<MemBlock tag:0x100 set_index:31>],
 '4007fc': [<MemBlock tag:0x100 set_index:31>

In [5]:
""" 按 set_index 分割内存块 只打印访问此 memory block 的信息 """

pprint(extract_mem_blocks(30, mem_block_dict))

{'400604': [],
 '400630': [],
 '40066c': [],
 '400708': [],
 '400724': [],
 '400730': [],
 '40073c': [],
 '400740': [],
 '40074c': [],
 '400754': [],
 '400758': [],
 '400770': [<MemBlock tag:0x100 set_index:30>],
 '40079c': [<MemBlock tag:0x100 set_index:30>],
 '4007c0': [],
 '4007d4': [],
 '4007fc': [],
 '40080c': [],
 '400848': [],
 '400864': [],
 '400870': [],
 '400874': []}


In [6]:
""" Fixpoint nodes basic generation. """
""" Illustrated with must analysis, set_index=31 as example. """
""" must分析例子, set_index=31 作为例子 也就是输出第31个set的must分析 这里在进行可视化时需要得到256个set的must分析 may分析 """

fp_nodes = fixpoint_nodes_generation(nodes=all_nodes, in_nodes=in_nodes, out_nodes=out_nodes,
                                     cache_config=cache_conf, evicted=False,
                                     mem_block_dict=extract_mem_blocks(31, mem_block_dict))
pprint(fp_nodes)

[<400604 ACC:[] IN:set(), OUT:{'400630', '400740'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400630 ACC:[] IN:{'400604'}, OUT:{'400730'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <40066c ACC:[] IN:{'400730'}, OUT:{'400708', '400724'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400708 ACC:[] IN:{'40066c'}, OUT:{'400874'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400724 ACC:[] IN:{'40066c'}, OUT:{'400730'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400730 ACC:[] IN:{'400630', '400724'}, OUT:{'40073c', '40066c'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <40073c ACC:[] IN:{'400730'}, OUT:{'400870'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400740 ACC:[] IN:{'400604'}, OUT:{'40074c', '400754'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <40074c ACC:[] IN:{'400740'}, OUT:{'400758'} IN_STAT:line:[{}, {}, {}, {}]

In [7]:
''' 上面是初始化 下面进行的是具体分析 注意这里需要entrynode信息 修改FixpointType即可 分别为 Must May Persistent 目前Persistent的输出未知, 继续测试即可; 这里的输出 e.g., 256为 memory block 的tag号, 因为已经指定了set_index 所以只需要tag即可 '''
fixpoint = Fixpoint(ty=FixpointType.Must, entry=entrynode, all_nodes=fp_nodes)
print(fixpoint.pse_topsort_seq)

fixpoint.run()
pprint(fp_nodes)

('400604', '400630', '400740', '400730', '40074c', '400754', '40073c', '40066c', '400758', '400758', '400870', '400708', '400724', '400864', '400864', '400874', '400874', '400770', '400770', '4007c0', '40079c', '4007c0', '40079c', '4007fc', '400848', '4007fc', '400848', '4007d4', '40080c', '4007d4', '40080c')
[<400604 ACC:[] IN:set(), OUT:{'400630', '400740'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400630 ACC:[] IN:{'400604'}, OUT:{'400730'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <40066c ACC:[] IN:{'400730'}, OUT:{'400708', '400724'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400708 ACC:[] IN:{'40066c'}, OUT:{'400874'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400724 ACC:[] IN:{'40066c'}, OUT:{'400730'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <400730 ACC:[] IN:{'400630', '400724'}, OUT:{'40073c', '40066c'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, 